# Recommender System part 2

## First part

In [1]:
from pymongo import MongoClient 
import pandas as pd
import sys
import numpy as np

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


In [3]:
try: 
    conn = MongoClient('192.168.1.72', 27017) 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 


Connected successfully!!!


#### reading the data from database

In [ ]:
mydb = conn.SK
collection1 = mydb.OrderMaster
%time df1 = pd.DataFrame(list(collection1.find({},{"Skcode":1,"orderDetails":1,"_id":0})))

In [ ]:
df1

#### extracting itemid's from orderDetails and exploding it using explode function

In [ ]:
df1["item"] = [[i[j]['ItemId'] for j in range(len(i))] for i in df1['orderDetails']]
df1 = df1[['Skcode','item']]
df2 = df1.explode('item')

In [ ]:
df2

In [ ]:
duplicate_bool = df2.duplicated(subset=['Skcode','item'], keep='last')
duplicate = df2.loc[duplicate_bool == True]

In [ ]:
duplicate

In [ ]:
# df2.loc[df2.item == 1]

#### Adding rating column in dataframe with value 1 and dropping the duplicate rows

In [ ]:
df2['rating'] = df2.apply(lambda _: '1', axis=1)
df2.columns = ['userID','itemID', 'rating']
df2

In [ ]:
df2.itemID.nunique()

In [ ]:
df2['rating'] = df2['rating'].astype(int)


In [ ]:
df3 = df2.drop_duplicates()


In [ ]:
df3

In [ ]:
df4 = df3.dropna()

#### calculating frequency by grouping rating on itemid and summing the list

In [ ]:
a=df2.groupby('itemID')['rating'].apply(list)
b = a.to_frame()
c = b.copy()
c

In [ ]:
c['frequency']  = [sum(i) for i in c['rating']]
c1 = c.reset_index()
c1

#### mapping itemid's and frequency and zipping them together

In [ ]:
#mapped = zip([i for i in c1['itemID']] , [j for j in c1['frequency']])
#lfreq = list(mapped)
#lfreq.append((322758,0))

In [ ]:
c2 = c1[['itemID','frequency']]

In [ ]:
lfreq = {}

In [ ]:
for index,row in c2.iterrows():
    lfreq[row['itemID']] = row['frequency']
    

In [ ]:
len(lfreq)

In [ ]:
#freq = [j for j in c1['frequency']]
#freq.append(0)

#### computing cosine similarity matrix

In [ ]:
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

In [ ]:
#testSubject = 'SK4005'


temp = []

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df4[['userID', 'itemID', 'rating']], reader)



k = 10

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()


In [ ]:
simsMatrix

In [ ]:
lFreVal = []

In [ ]:
lc = list(trainSet._raw2inner_id_items)

In [ ]:
for i in lc:
    lFreVal.append(lfreq[i])

In [32]:
simsMatrix1 = lFreVal * simsMatrix

In [33]:
simsMatrix1

array([[173., 573., 351., ...,   0.,   0.,   0.],
       [173., 573., 351., ...,   0.,   0.,   0.],
       [173., 573., 351., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   1.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   1.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   1.]])

In [34]:
#l = [i for i in c1['itemID'] ]
#l.append(1000000000)

In [35]:
df = pd.DataFrame(simsMatrix1 , columns = lc, index = lc)

In [36]:
df

,1576,923,933,1197,1784,481,4315,3961,3960,1464,...,245809,261036,202635,212520,213148,15990,178227,135060,329351,208517
1576,173.0,573.0,351.0,1483.0,227.0,525.0,209.0,275.0,151.0,747.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
923,173.0,573.0,351.0,1483.0,227.0,525.0,0.0,0.0,0.0,747.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
933,173.0,573.0,351.0,1483.0,227.0,525.0,0.0,0.0,0.0,747.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,173.0,573.0,351.0,1483.0,227.0,525.0,209.0,275.0,151.0,747.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1784,173.0,573.0,351.0,1483.0,227.0,525.0,209.0,275.0,151.0,747.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
178227,0.0,0.0,0.0,1483.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
135060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
329351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [55]:
df.to_pickle('CosSimMatrix.pkl')  

In [30]:
df_new = pd.read_pickle('CosSimMatrix.pkl')


In [1]:
#df_dic = df.to_json(orient='index')
#len(df_dic)
#df_dic = df.to_dict('index')

In [29]:
lcolumn = list(df.columns)

In [30]:
mapped1 = zip(lcolumn , freq)
lis = list(mapped1)

In [31]:
try: 
    conn1 = MongoClient('192.168.1.101', 27017) 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 


Connected successfully!!!


In [39]:
mydb1 = conn.ankit_database
collection2 = mydb1.RecSysItem

In [2]:
#db.insert_many(df.to_dict("records"))
# Insert Data 
#collection2.insert_many(df.to_dict()) 
  

In [ ]:
#collection = conn.rough
  
#emp_rec1 = { 
#        "name":"Mr.Geek", 
#        "eid":24, 
#        "location":"delhi"
#        } 
#emp_rec2 = { 
#        "name":"Mr.Shaurya", 
#        "eid":14, 
#        "location":"delhi"
#        } 

# Insert Data 
# rec_id1 = collection.insert_one(emp_rec1) 
# rec_id2 = collection.insert_one(emp_rec2) 
  


# Second Part

In [35]:
itemid = 2
kNeighbors = [(itemid,1.0)]

In [56]:
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    #similarityRow = df.loc[itemID].values
    mapped1 = zip(lcolumn , df.loc[itemID])
    lis = list(mapped1)
    for itemid, score in lis:
        candidates[itemid] += score * (rating )

pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    print(itemID,ratingSum)
    pos += 1
    if (pos > 10):
        break



164 5033.0
669 4695.0
1294 3999.0
436 3968.0
1197 3904.0
1196 3745.0
205 3332.0
204 3149.0
561 2997.0
911 2744.0
480 2653.0


In [ ]:
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = df[itemID].values
    mapped = zip(lcolumn , df[itemID])
    lis = list(mapped)
    for itemid, score in lis:
        candidates[innerID] += score * (rating )

        
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    print(itemID,ratingSum)
    pos += 1
    if (pos > 10):
        break

        


In [ ]:
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)
 

# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

    
